Import a necessary libraries:

In [1]:
import pandas as pd
import numpy as np
import nltk as nl
import string as st
nl.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Deniz\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Download data set and creata data frame for analyse:

In [2]:
data_frame = pd.read_csv("Comments.csv", sep = ',')

data_frame

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


Read about data frame:

In [3]:
print(data_frame.columns)

data_frame.info

Index(['comment', 'toxic'], dtype='object')


<bound method DataFrame.info of                                                  comment  toxic
0                   Верблюдов-то за что? Дебилы, бл...\n    1.0
1      Хохлы, это отдушина затюканого россиянина, мол...    1.0
2                              Собаке - собачья смерть\n    1.0
3      Страницу обнови, дебил. Это тоже не оскорблени...    1.0
4      тебя не убедил 6-страничный пдф в том, что Скр...    1.0
...                                                  ...    ...
14407  Вонючий совковый скот прибежал и ноет. А вот и...    1.0
14408  А кого любить? Гоблина тупорылого что-ли? Или ...    1.0
14409  Посмотрел Утомленных солнцем 2. И оказалось, ч...    0.0
14410  КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...    1.0
14411  До сих пор пересматриваю его видео. Орамбо кст...    0.0

[14412 rows x 2 columns]>

Convert target column to int for simplification:

In [4]:
data_frame["toxic"] = data_frame["toxic"].apply(int)

data_frame["toxic"]

0        1
1        1
2        1
3        1
4        1
        ..
14407    1
14408    1
14409    0
14410    1
14411    0
Name: toxic, Length: 14412, dtype: int64

Check fullness of data frame:

In [5]:
data_frame.isna().count()

comment    14412
toxic      14412
dtype: int64

Check distribution:

In [6]:
data_frame["toxic"].value_counts()

0    9586
1    4826
Name: toxic, dtype: int64

Check examples of toxic comments:

In [7]:
for comment in data_frame[data_frame["toxic"] == 1]["comment"].head(5):
    print(comment)

Верблюдов-то за что? Дебилы, бл...

Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.

Собаке - собачья смерть

Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?

тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)



Check examples of non-toxic comments:

In [8]:
for comment in data_frame[data_frame["toxic"] == 0]["comment"].head(5):
    print(comment)

В шапке были ссылки на инфу по текущему фильму марвел. Эти ссылки были заменены на фразу Репортим брипидора, игнорируем его посты. Если этого недостаточно, чтобы понять, что модератор абсолютный неадекват, и его нужно лишить полномочий, тогда эта борда пробивает абсолютное дно по неадекватности.

Почитайте посты у этого автора,может найдете что нибудь полезное. Надеюсь помог) https: pikabu.ru story obyichnyie budni dezsluzhbyi 4932098

Про графику было обидно) я так то проходил все серии гта со второй части по пятую, кроме гта 4. И мне не мешала графика ни в одной из частей. На компе у меня было куча видеокарт. Начиная с 32мб RIVA TNT и заканчивая 2Гб 560Ti на которой я спокойно играю который год в танки, гта5, ведьмака3 купил на распродаже и начал проходить. Да, не на ультрах. С пониженными текстурами. И не мешает. Я не понимаю дрочева на графике, требовать графику уровня плойки 4 минимум. Мне надо чтобы глаза не резало, только и всего. По поводу управления, мне не хватает переходника

The last example of non-toxic comments is incorrect that means our data labeling no such good, and our model cant reach perfect results.

Divide our data set to train and test parts:

In [9]:
train_data_frame, test_data_frame = train_test_split(data_frame, test_size = 750)

Check value of test and train data frames:

In [10]:
print(test_data_frame["toxic"].value_counts(0))

test_data_frame

0    484
1    266
Name: toxic, dtype: int64


,comment,toxic
10207,"Мне кажется, что ритуал экзорцизма обойдется д...",0
9707,В америке еще больше возросло число заказов на...,0
7289,"А это где? У нас тоже чистили, даже 2 или три ...",0
2693,А ты думаешь почему к власти нацисты пришли? Г...,1
4799,Ну слава тебе господи я не одна такая в Минске...,1
...,...,...
5510,Так самсунг продали компании Hanwha. Причем пр...,0
8231,Да и так видно что СЭС не эффективна по вложен...,0
9889,"Проблема в объективной реальности, такого поня...",0
11128,"работать с крупными брендами, делать для них в...",0


In [11]:
print(train_data_frame["toxic"].value_counts(0))

train_data_frame

0    9102
1    4560
Name: toxic, dtype: int64


,comment,toxic
33,С каких пор неадекват - это оскорбление? На Пи...,1
273,Я так пишу потому что делаю это с телефона. И ...,1
13974,Как Навальный и его петушня? Не дождёшься\n,1
8168,А кто не захочет таких кукол брать- судить за ...,1
12681,Предусматривалась возможность сбрасывания бомб...,0
...,...,...
13541,"Дом.рф - это государственная контора, бывшее А...",0
6616,Да какой он полноценный? Аренда видимо дорогая...,0
2439,ты чо дурачочек чтоле\n,1
10783,"У тебя бутылка в жопе, что ли, раз ты такой об...",1


We will use logistic regression.

At first we need to transform our data frames to vectors. Its reached by separation on tokens (without symbols and stop words).

At first create list of stop word and create stemming prototype.

Create funstions for tokenize:

In [12]:
#russian_stop_words = stopwords.words("russian") # doesn't work idk
snowball = SnowballStemmer(language = "russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language = "russian")
    tokens = [i for i in tokens if i not in st.punctuation]
    
    #if remove_stop_words:
    #    tokens = [i for i in tokens if i not in russian_stop_words]
    
    tokens = [snowball.stem(i) for i in tokens]
    
    return tokens

We will use TFID method:

In [13]:
vectorizer = TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words = True))

Learn vectorizer:

In [14]:
features = vectorizer.fit_transform(train_data_frame["comment"])

C:\ProgramData\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


`fit_transoform()` returns prepare features for model.

In [15]:
model = LogisticRegression(random_state = 0) # Create a class of logistic regression
model.fit(features, train_data_frame["toxic"])

LogisticRegression(random_state=0)

Check results of models work:

In [16]:
model.predict(features[0])

train_data_frame["comment"].iloc[0]

'С каких пор неадекват - это оскорбление? На Пикабу тысячи таких комментариев, и модераторы, даже будучи призванными, их не удаляют. Я вас дураком не называл. Это был вопрос: ты дурак? . И опять же, таких вопросов на Пикабу тысячи, если не больше, вплоть до ты ебанутый? или ты долбоеб? - и они тоже не считаются оскорблением и не удаляются. Вы считаете оскорблением то, что тысячи раз до этого оскорблениями не считали. Если правила в одном случае работают, а в другом нет, то это что угодно, но не правила.\n'

Create a pipeline for analyse input data:

In [17]:
model_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))),
    ("model", LogisticRegression(random_state=0))
]
)

Train pipeline:

In [18]:
model_pipeline.fit(train_data_frame["comment"], train_data_frame["toxic"])

C:\ProgramData\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x00000194CA3589D0>)),
                ('model', LogisticRegression(random_state=0))])

Use any text:

In [19]:
model_pipeline.predict(["Привет, у меня всё в порядке!"])

array([0], dtype=int64)

In [20]:
model_pipeline.predict(["Козёл ты этакий!"])

array([1], dtype=int64)

Check metrics values.

At first check presicion:

In [21]:
precision_score(y_true = test_data_frame["toxic"], y_pred = model_pipeline.predict(test_data_frame["comment"]))

0.8911917098445595

In [22]:
recall_score(y_true = test_data_frame["toxic"], y_pred = model_pipeline.predict(test_data_frame["comment"]))

0.6466165413533834

These values not correct for us. So let's change recall and precision values:

In [23]:
prec, rec, thresholds = precision_recall_curve(y_true = test_data_frame["toxic"], probas_pred = model_pipeline.predict_proba(test_data_frame["comment"])[: , 1])

Draw precision and recall dependence:

In [24]:
#Doesn't works beacuse old version library.
#plot_precision_recall_curve(estimator = model_pipeline, X = test_data_frame["comment"], y = test_data_frame["toxic"])

Check values where `precision > 0,95`:

In [25]:
np.where(prec > 0.95)

(array([684, 685, 686, 687, 697, 698, 699, 700, 701, 702, 703, 704, 705,
        706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718,
        719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730, 731,
        732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744,
        745, 746, 747, 748], dtype=int64),)

In [26]:
thresholds[374]

0.2398092337036729

In [27]:
precision_score(y_true = test_data_frame["toxic"], y_pred = model_pipeline.predict_proba(test_data_frame["comment"])[: , 1] > thresholds[374])


0.6666666666666666

In [28]:
recall_score(y_true = test_data_frame["toxic"], y_pred = model_pipeline.predict_proba(test_data_frame["comment"])[: , 1] > thresholds[374])

0.9398496240601504

We ge small recall.
Let's try to upgrade our model. For that create new pipeline with hyper-parametr:

In [29]:
grid_pipeline = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words = True))),
    ("model", 
        GridSearchCV(
            LogisticRegression(random_state = 0),
            param_grid = {'C': [0.1, 1, 10.]}, # Check which parametr better
            cv = 3,
            verbose = 4 # Display all information about learning
        )
    )
])

In [30]:
grid_pipeline.fit(train_data_frame["comment"], train_data_frame["toxic"])

C:\ProgramData\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV 1/3] END .............................C=0.1;, score=0.716 total time=   0.3s
[CV 2/3] END .............................C=0.1;, score=0.721 total time=   0.2s
[CV 3/3] END .............................C=0.1;, score=0.714 total time=   0.3s
[CV 1/3] END ...............................C=1;, score=0.841 total time=   0.5s
[CV 2/3] END ...............................C=1;, score=0.864 total time=   0.6s
[CV 3/3] END ...............................C=1;, score=0.857 total time=   0.8s


C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/3] END ............................C=10.0;, score=0.867 total time=   0.9s


C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/3] END ............................C=10.0;, score=0.883 total time=   1.0s


C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/3] END ............................C=10.0;, score=0.874 total time=   1.0s


C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x00000194CA348040>)),
                ('model',
                 GridSearchCV(cv=3,
                              estimator=LogisticRegression(random_state=0),
                              param_grid={'C': [0.1, 1, 10.0]}, verbose=4))])

The best variant is with c = 10:

In [31]:
model_pipeline_c_10 = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer = lambda x: tokenize_sentence(x, remove_stop_words = True))),
    ("model", LogisticRegression(random_state = 0, C = 10.))
])

In [32]:
model_pipeline_c_10.fit(train_data_frame["comment"], train_data_frame["toxic"])

C:\ProgramData\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\ProgramData\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(tokenizer=<function <lambda> at 0x00000194CB6412D0>)),
                ('model', LogisticRegression(C=10.0, random_state=0))])

In [33]:
prec_c_10, rec_c_10, thresholds_c_10 = precision_recall_curve(y_true = test_data_frame["toxic"], probas_pred = model_pipeline_c_10.predict_proba(test_data_frame["comment"])[:, 1])

In [34]:
np.where(prec_c_10 > 0.95)

(array([640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652,
        653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665,
        666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678,
        679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691,
        692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704,
        705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717,
        718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 728, 729, 730,
        731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743,
        744, 745, 746, 747, 748], dtype=int64),)

Calculate precision and recall for the best variant:

In [35]:
precision_score(y_true = test_data_frame["toxic"], y_pred = model_pipeline_c_10.predict_proba(test_data_frame["comment"])[: , 1] > thresholds_c_10[316])

0.6027713625866051

In [36]:
recall_score(y_true = test_data_frame["toxic"], y_pred = model_pipeline_c_10.predict_proba(test_data_frame["comment"])[: , 1] > thresholds_c_10[316])


0.981203007518797

After this you can deploy this model in production. But in the last time more useful NLP types.